In [65]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

Import all data as one dataframe

In [34]:
data_dir = 'trainingdata_new'
all_data = glob.glob(os.path.join(data_dir, "*.csv"))
combined_data = pd.concat([pd.read_csv(file) for file in all_data], ignore_index=True)
combined_data.drop(["Unnamed: 0"], axis = 1)

,Wavelength,V1,V2,Gain,Out
0,510,0.050000,0.050000,4,0.045723
1,510,0.130612,0.050000,4,0.049555
2,510,0.211224,0.050000,4,0.049945
3,510,0.291837,0.050000,4,0.050324
4,510,0.372449,0.050000,4,0.050414
...,...,...,...,...,...
1395,590,3.677551,0.372449,4,0.061949
1396,590,3.758163,0.291837,4,0.062184
1397,590,3.838776,0.211224,4,0.063187
1398,590,3.919388,0.130612,4,0.063203


Add new column that adjusts output voltage for the gain factor

In [40]:
combined_data.loc[0,'Out']
new_out = []
for i in range(len(combined_data)):
    adj_out = combined_data.loc[i,'Out']*combined_data.loc[i,'Gain']
    new_out.append(adj_out)
combined_data['Gain Volts'] = new_out

,Wavelength,V1,V2,Gain,Out,Gain Volts,Volt Pairs
0,510,0.050000,0.050000,4,0.045723,0.182891,"(0.05, 0.05)"
1,510,0.130612,0.050000,4,0.049555,0.198219,"(0.1306122448979592, 0.05)"
2,510,0.211224,0.050000,4,0.049945,0.199781,"(0.2112244897959184, 0.05)"
3,510,0.291837,0.050000,4,0.050324,0.201297,"(0.2918367346938775, 0.05)"
4,510,0.372449,0.050000,4,0.050414,0.201656,"(0.3724489795918367, 0.05)"
...,...,...,...,...,...,...,...
1395,590,3.677551,0.372449,4,0.061949,0.247797,"(3.6775510204081634, 0.3724489795918367)"
1396,590,3.758163,0.291837,4,0.062184,0.248734,"(3.7581632653061225, 0.2918367346938775)"
1397,590,3.838776,0.211224,4,0.063187,0.252750,"(3.8387755102040817, 0.2112244897959184)"
1398,590,3.919388,0.130612,4,0.063203,0.252812,"(3.919387755102041, 0.1306122448979592)"


Fitting model as shown in svm_regression_test.ipynb (will go into further detail)

In [68]:
x = combined_data['V1'].to_numpy()
y = combined_data['V2'].to_numpy()
z = combined_data['Gain Volts'].to_numpy(dtype = 'object')

X = np.column_stack((x, y))

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1]}
grid_search = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=5)
grid_search.fit(X, z)
best_c = grid_search.best_params_['C']
best_gamma = grid_search.best_params_['gamma']

model = SVR(kernel='rbf', C=best_c, gamma=best_gamma) 
model.fit(X,z)
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1 
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 500),
                       np.linspace(x2_min, x2_max, 500))
xx_input = np.column_stack((xx1.ravel(), xx2.ravel()))
y_pred = model.predict(xx_input)

# Attempting Neural Network Fitting
The above fit is a bit rough, and part of the problem is that it's wavelength agnostic. Of course, we do have a wavelength dependency in our data. However, the reason it's wavelength agnostic is because (from what I can gather) getting the predictions we'd like from a 4D SVM regressor is difficult. The current goal is to give two inputs (wavelength and desired polarization angle) and receive the input voltages needed for the LCVR's to produce the angle. Therefore a neural net may be more apt to fit this data.

In [ ]:
## Test neural network code created with Gemini

import tensorflow as tf
from tensorflow.keras import layers

data = tf.convert_to_tensor(combined_data)

# Split into input (X) and output (y)
X = data[:, :4]
y = data[:, 4:]

# Split into training and validation sets
# ... 

# Model Definition
model = tf.keras.Sequential([
  layers.Dense(10, activation='relu', input_shape=(4,)),  # Hidden layer
  layers.Dense(2)  # Output layer with 2 neurons
])

# Compilation
model.compile(optimizer='adam', loss='mean_squared_error')

# Training
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val)) 

# Prediction
new_input = np.array([[1, 2, 3, 4]])  # Input with 2 known features
prediction = model.predict(new_input)
print(prediction) 
